In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT/data/motion3d')

In [2]:
def is_consecutive(lst):
    # 빈 리스트나 하나의 요소만 있는 리스트는 연속적이라고 간주
    if len(lst) <= 1:
        return True
    
    # 리스트를 정렬
    sorted_lst = sorted(lst)
    
    # 정렬된 리스트의 각 요소가 이전 요소보다 1씩 큰지 확인
    for i in range(1, len(sorted_lst)):
        if sorted_lst[i] != sorted_lst[i - 1] + 1:
            return False
    
    return True

def check_keys(pkl):
    for data_type in pkl.keys():
        print(data_type)
        for key in pkl[data_type].keys():
            if type(pkl[data_type][key]) is np.ndarray:
                print(key, type(pkl[data_type][key]), pkl[data_type][key].shape)
            elif type(pkl[data_type][key]) is list:
                print(key, type(pkl[data_type][key]), len(pkl[data_type][key]))
            else:
                print(key, type(pkl[data_type][key]))
                
def load_data_dict(dataset):
    dataset_path = f'{dataset}.pkl'
    dict_path = f'{dataset}_dict.pkl'
    if os.path.exists(dict_path):
        data_dict = readpkl(dict_path)
        print(f'{dict_path} exists')
    else:
        # Load h36m data
        pkl = readpkl(dataset_path)
        # check the keys
        #check_keys(pkl)
        # Create a dictionary for h36m_gt
        data_dict = {}

        for data_type in ['train', 'test']:
            sources = set(pkl[data_type]['source'])
            frames = pkl[data_type]['frame']
            for source in tqdm(sources):
                source_frames = np.where(pkl[data_type]['source'] == source)[0]
                assert is_consecutive(source_frames), f'Frames are not consecutive: {source}'
                #print(len(source_frames))
                if 'fit3d' in dataset:
                    splited = source.split('_')
                    subject = splited[1]
                    cam_id = splited[2]
                    action = '_'.join(splited[3:])
                elif '3dhp' in dataset:
                    splited = source.split('_')
                    subject = splited[0]
                    cam_id = splited[0]
                    action = splited[0]
                else:
                    subject, cam_id, action = source.split('_')
                if subject not in data_dict:
                    data_dict[subject] = {}
                if action not in data_dict[subject]:
                    data_dict[subject][action] = {}
                if cam_id not in data_dict[subject][action]:
                    data_dict[subject][action][cam_id] = {
                        'joint_2d': pkl[data_type]['joint_2d'][source_frames],
                        'cam_param': pkl[data_type]['cam_param'][source_frames[0]]
                    } 
                    if 'same_z' in dataset:
                        data_dict[subject][action][cam_id]['joint_2d'] = pkl[data_type]['joint_2d_from_canonical_3d'][source_frames]
                        data_dict[subject][action][cam_id]['world_3d'] = pkl[data_type]['world_3d_from_canonical_3d'][source_frames]
                        data_dict[subject][action][cam_id]['cam_3d'] = pkl[data_type]['cam_3d_from_canonical_3d'][source_frames]
                    else:
                        data_dict[subject][action][cam_id]['joint_2d'] = pkl[data_type]['joint_2d'][source_frames]
                        data_dict[subject][action][cam_id]['world_3d'] = pkl[data_type]['world_3d'][source_frames]
                        data_dict[subject][action][cam_id]['cam_3d'] = pkl[data_type]['cam_3d'][source_frames]
                        
        savepkl(data_dict, dict_path)
        del pkl

    return data_dict

In [3]:
os.listdir()

['fit3d_gt_pkl_canonical_3d_same_z',
 'fit3d_gt_canonical_3d_same_z_all_test.pkl',
 'h36m_gt_canonical_3d_same_dist_s15678_tr_54138969_ts_others.pkl',
 'h36m_gt_pkl_canonical_3d_same_dist',
 'fit3d_gt_pkl',
 '3dhp_gt_train_pkl',
 'h36m_gt_pkl_canonical_3d_fixed_dist',
 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678.pkl',
 '3dhp_gt_test_TS1_4.pkl',
 'h36m_gt_canonical_3d_same_dist_tr_s1_ts_s5678.pkl',
 'fit3d_all_train_except_target_action.pkl',
 'h36m_sh_conf_cam_source_final.pkl',
 'h36m_gt_canonical_3d_same_z.pkl',
 '3dhp_gt_test_all_train.pkl',
 'MB3D_f243s81',
 'h36m_gt_dict.pkl',
 'fit3d_gt_all_test.pkl',
 'h36m_gt_s15678_tr_54138969_ts_others.pkl',
 'fit3d_gt_canonical_3d_same_z_all_test_dict.pkl',
 'random_limb_pkl',
 'h36m_gt.pkl',
 'h36m_gt_canonical_3d_same_dist.pkl',
 'h36m_gt_canonical_3d_fixed_dist_tr_s1_ts_s5678.pkl',
 '3dhp_gt_test_pkl_prev',
 'h36m_gt_test_trans-0.5,0,-0.5_pkl',
 'h36m_gt_pkl_canonical_3d_same_z',
 'h36m_gt_canonical_3d_same_z_dict.pkl',
 '3dhp_gt_test_pkl

### Prepare plot

In [5]:
import matplotlib.patches as patches

# world 3d
fig_world_3d = plt.figure(0)
fig_world_3d.clear()
ax_world_3d = axes_3d(fig_world_3d, loc=121, zlim=(0, 2), xlim=(-3, 3), ylim=(-5, 5), view=(90, 0))
ax_world_3d_2 = axes_3d(fig_world_3d, loc=122, zlim=(0, 2), xlim=(-3, 3), ylim=(-5, 5), view=(0, 0))
ax_world_3d.set_title(f'wrt world_frame')
fig_world_3d.tight_layout()

# cam 3d
fig_cam_3d = plt.figure(1, figsize=(10, 6))
fig_cam_3d.clear()
# Figure 외곽선 추가
rect = patches.Rectangle((0, 0), 1, 1, transform=fig_cam_3d.transFigure,
                         linewidth=2, edgecolor='r', facecolor='none')
fig_cam_3d.patches.append(rect)
axd = fig_cam_3d.subplot_mosaic(
    """
    AB
    CB
    """,
    per_subplot_kw={('A'): {'projection': '3d'}}
)

ax_cam_3d = axes_3d(ax=axd['A'], zlim=(0,5), view=(-90, -90))
ax_cam_3d.set_title(f'wrt cam_frame')
ax_cam_xz = axd['B'] 
ax_cam_xz.set_xlim(-2, 2)
ax_cam_xz.set_ylim(0, 8)
#fig_cam_3d.add_subplot([0.5, 0, 0.5, 0.5], xlim=(-2, 2), ylim=(0, 8)) # x, z
ax_cam_xz.set_aspect('equal')
ax_cam_xz.set_xlabel('x')
ax_cam_xz.set_ylabel('z')
ax_cam_xz.set_title(f'x-z plane')
ax_cam_zy = axd['C'] #fig_cam_3d.add_subplot(133, xlim=(8, 0), ylim=(2, -2)) # y, z
ax_cam_zy.set_xlim(8, 0)
ax_cam_zy.set_ylim(2, -2)
ax_cam_zy.set_aspect('equal')
ax_cam_zy.set_xlabel('z')
ax_cam_zy.set_ylabel('y')
ax_cam_zy.set_title(f'z-y plane')
fig_cam_3d.tight_layout()

# image 2d
fig_img_2d = plt.figure(2)
fig_img_2d.clear()
ax_img_2d = axes_2d(fig_img_2d, W=900, H=900)
ax_img_2d.set_title(f'wrt image frame')

# cam_3d_rootrel
fig_cam_3d_rootrel = plt.figure(3, figsize=(10, 6))
fig_cam_3d_rootrel.clear()
# Figure 외곽선 추가
rect = patches.Rectangle((0, 0), 1, 1, transform=fig_cam_3d_rootrel.transFigure,
                         linewidth=2, edgecolor='r', facecolor='none')
fig_cam_3d_rootrel.patches.append(rect)
axd_rootrel = fig_cam_3d_rootrel.subplot_mosaic(
    """
    AB
    CB
    """,
    per_subplot_kw={('A'): {'projection': '3d'}}
)

ax_cam_3d_rootrel = axes_3d(ax=axd_rootrel['A'], zlim=(0,5), view=(-90, -90))
ax_cam_3d_rootrel.set_title(f'wrt cam_frame')
ax_cam_xz_rootrel = axd_rootrel['B'] 
ax_cam_xz_rootrel.set_xlim(-2, 2)
ax_cam_xz_rootrel.set_ylim(-2, 2)
ax_cam_xz_rootrel.set_aspect('equal')
ax_cam_xz_rootrel.set_xlabel('x')
ax_cam_xz_rootrel.set_ylabel('z')
ax_cam_xz_rootrel.set_title(f'x-z plane')
ax_cam_zy_rootrel = axd_rootrel['C'] 
ax_cam_zy_rootrel.set_xlim(2, -2)
ax_cam_zy_rootrel.set_ylim(2, -2)
ax_cam_zy_rootrel.set_aspect('equal')
ax_cam_zy_rootrel.set_xlabel('z')
ax_cam_zy_rootrel.set_ylabel('y')
ax_cam_zy_rootrel.set_title(f'z-y plane')
fig_cam_3d_rootrel.tight_layout()

linewidth_2d = 0.1

### H36M

In [6]:
h36m_gt_dict = load_data_dict('h36m_gt')
h36m_canonical_gt_dict = load_data_dict('h36m_gt_canonical_3d_same_z')

h36m_gt_dict.pkl exists
h36m_gt_canonical_3d_same_z_dict.pkl exists


In [11]:
dataset = 'h36m_gt'

if dataset == 'h36m_gt':
    data_dict = h36m_gt_dict
elif dataset == 'h36m_gt_canonical_3d_same_z':
    data_dict = h36m_canonical_gt_dict
folder = f'/home/hrai/codes/MotionBERT/custom_codes/Dataset_analysis/{dataset}'
if not os.path.exists(folder):
    os.makedirs(folder)


subject_list = ['S1']
#subject_list = ['S5', 'S6', 'S7', 'S8']
#subject_list = ['S1', 'S5', 'S6', 'S7', 'S8']
#subject_list = ['S9', 'S11'] 
#subject_list = ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11']

keypoint_list = ['pelvis', 'r_wrist', 'l_wrist', 'r_ankle', 'l_ankle']
keypoint_idxs = [get_h36m_keypoint_index(keypoint) for keypoint in keypoint_list]

clear_axes([ax_world_3d, ax_world_3d_2, ax_cam_3d, ax_cam_xz, ax_cam_zy, ax_img_2d])

ax_world_3d.set_title(f'wrt world_frame {subject_list} {keypoint_list}')
ax_cam_3d.set_title(f'wrt cam_frame {subject_list} {keypoint_list}')
ax_cam_xz.set_title(f'cam x-z plane')
ax_cam_zy.set_title(f'cam z-y plane')
ax_img_2d.set_title(f'wrt image frame \n{subject_list} {keypoint_list}')

# collect traj points
camera_dict = {}
for subject in subject_list:
    action_list = list(data_dict[subject].keys())
    camera_dict = {}
    for action in action_list:
        cam_list = list(data_dict[subject][action].keys())
        for cam_id, color in zip(cam_list, ['r', 'b', 'k', 'y']):
            joint_2d = data_dict[subject][action][cam_id]['joint_2d']
            cam_3d = data_dict[subject][action][cam_id]['cam_3d']
            cam_3d_hat = cam_3d - cam_3d[:, 0:1, :]
            for key_idx in keypoint_idxs:
                ax_cam_3d.plot(cam_3d[:, key_idx, 0], cam_3d[:, key_idx, 1], cam_3d[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_xz.plot(cam_3d[:, key_idx, 0], cam_3d[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_zy.plot(cam_3d[:, key_idx, 2], cam_3d[:, key_idx, 1], color=color, linewidth=linewidth_2d)
                ax_img_2d.plot(joint_2d[:, key_idx, 0], joint_2d[:, key_idx, 1], color=color, linewidth=linewidth_2d, label=f'{cam_id}')
                ax_cam_3d_rootrel.plot(cam_3d_hat[:, key_idx, 0], cam_3d_hat[:, key_idx, 1], cam_3d_hat[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_xz_rootrel.plot(cam_3d_hat[:, key_idx, 0], cam_3d_hat[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_zy_rootrel.plot(cam_3d_hat[:, key_idx, 2], cam_3d_hat[:, key_idx, 1], color=color, linewidth=linewidth_2d)
            
            # generate camera frame
            if cam_id not in camera_dict:
                cam_param = data_dict[subject][action][cam_id]['cam_param'].tolist()
                intrinsic = cam_param['intrinsic']
                extrinsic = cam_param['extrinsic']
                R = extrinsic[:3, :3]
                t = extrinsic[:3, 3]
                C = -R.T @ t
                camera = Camera(origin=C, 
                                calib_mat=intrinsic, 
                                cam_default_R=R, 
                                IMAGE_HEIGHT=1000, 
                                IMAGE_WIDTH=1000,
                                cam_name=cam_id)
                camera_dict[cam_id] = camera
            
        world_3d = data_dict[subject][action][cam_id]['world_3d']
        world_3d_traj = world_3d[:, keypoint_idxs, :]
        for key_idx in keypoint_idxs:
            if key_idx == 0: 
                color = 'y'
                linewidth = 0.1
            else: 
                color = 'k'
                linewidth = 0.05
            ax_world_3d.plot(world_3d[:, key_idx, 0], world_3d[:, key_idx, 1], world_3d[:, key_idx, 2], label=f'{subject}', color=color, linewidth=linewidth)
            ax_world_3d_2.plot(world_3d[:, key_idx, 0], world_3d[:, key_idx, 1], world_3d[:, key_idx, 2], label=f'{subject}', color=color, linewidth=linewidth)
        
        
handles, labels = ax_img_2d.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
leg = ax_img_2d.legend(by_label.values(), by_label.keys())

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(4.0)
        
# plot traj
dpi = 500

plt.sca(ax_world_3d)
for key in camera_dict.keys():
    camera_dict[key].cam_frame.draw3d()
#plt.show()
plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_world_3d.png', dpi=dpi)

plt.sca(ax_cam_3d)
#plt.show()
plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_cam_3d.png', dpi=dpi)

plt.sca(ax_img_2d)
#plt.show()
plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_img_2d.png', dpi=dpi)

plt.sca(ax_cam_3d_rootrel)
#plt.show()
plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_cam_3d_rootrel.png', dpi=dpi)

### Fit3D

In [6]:
fit3d_gt_dict = load_data_dict('fit3d_gt_all_test')
fit3d_canonical_gt_dict = load_data_dict('fit3d_gt_canonical_3d_same_z_all_test')

fit3d_gt_all_test_dict.pkl exists
fit3d_gt_canonical_3d_same_z_all_test_dict.pkl exists


In [9]:
dataset = 'fit3d_gt_canonical_3d_same_z_all_test' # 'fit3d_gt_all_test', 'fit3d_gt_canonical_3d_same_z_all_test'

if dataset == 'fit3d_gt_all_test':
    data_dict = fit3d_gt_dict
elif dataset == 'fit3d_gt_canonical_3d_same_z_all_test':
    data_dict = fit3d_canonical_gt_dict
folder = f'/home/hrai/codes/MotionBERT/custom_codes/Dataset_analysis/{dataset}'
if not os.path.exists(folder):
    os.makedirs(folder)

subject_list = data_dict.keys()
keypoint_list = ['pelvis', 'r_wrist', 'l_wrist', 'r_ankle', 'l_ankle']
# keypoint_list = ['pelvis']
# keypoint_list = ['r_wrist', 'l_wrist']
# keypoint_list = ['r_ankle', 'l_ankle']
keypoint_idxs = [get_h36m_keypoint_index(keypoint) for keypoint in keypoint_list]

clear_axes([ax_world_3d, ax_world_3d_2, ax_cam_3d, ax_cam_xz, ax_cam_zy, ax_img_2d])

ax_world_3d.set_title(f'wrt world_frame {subject_list} {keypoint_list}')
ax_cam_3d.set_title(f'wrt cam_frame {subject_list} {keypoint_list}')
ax_cam_xz.set_title(f'cam x-z plane')
ax_cam_zy.set_title(f'cam z-y plane')
ax_img_2d.set_title(f'wrt image frame \n{subject_list} {keypoint_list}')

# collect traj points
camera_dict = {}
for subject in subject_list:
    action_list = list(data_dict[subject].keys())
    camera_dict = {}
    for action in action_list:
        cam_list = list(data_dict[subject][action].keys())
        for cam_id, color in zip(cam_list, ['r', 'b', 'k', 'y']):
            joint_2d = data_dict[subject][action][cam_id]['joint_2d']
            cam_3d = data_dict[subject][action][cam_id]['cam_3d']
            cam_3d_hat = cam_3d - cam_3d[:, 0:1, :]
            for key_idx in keypoint_idxs:
                ax_cam_3d.plot(cam_3d[:, key_idx, 0], cam_3d[:, key_idx, 1], cam_3d[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_xz.plot(cam_3d[:, key_idx, 0], cam_3d[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_zy.plot(cam_3d[:, key_idx, 2], cam_3d[:, key_idx, 1], color=color, linewidth=linewidth_2d)
                ax_img_2d.plot(joint_2d[:, key_idx, 0], joint_2d[:, key_idx, 1], color=color, linewidth=linewidth_2d, label=f'{cam_id}')
                ax_cam_3d_rootrel.plot(cam_3d_hat[:, key_idx, 0], cam_3d_hat[:, key_idx, 1], cam_3d_hat[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_xz_rootrel.plot(cam_3d_hat[:, key_idx, 0], cam_3d_hat[:, key_idx, 2], color=color, linewidth=linewidth_2d)
                ax_cam_zy_rootrel.plot(cam_3d_hat[:, key_idx, 2], cam_3d_hat[:, key_idx, 1], color=color, linewidth=linewidth_2d)
            
            # generate camera frame
            if cam_id not in camera_dict:
                cam_param = data_dict[subject][action][cam_id]['cam_param'].tolist()
                intrinsic = cam_param['intrinsic']
                extrinsic = cam_param['extrinsic']
                R = extrinsic[:3, :3]
                t = extrinsic[:3, 3]
                C = -R.T @ t
                camera = Camera(origin=C, 
                                calib_mat=intrinsic, 
                                cam_default_R=R, 
                                IMAGE_HEIGHT=900, 
                                IMAGE_WIDTH=900,
                                cam_name=cam_id)
                camera_dict[cam_id] = camera
            
        world_3d = data_dict[subject][action][cam_id]['world_3d']
        world_3d_traj = world_3d[:, keypoint_idxs, :]
        for key_idx in keypoint_idxs:
            if key_idx == 0: 
                color = 'y'
                linewidth = 0.1
            else: 
                color = 'k'
                linewidth = 0.05
            ax_world_3d.plot(world_3d[:, key_idx, 0], world_3d[:, key_idx, 1], world_3d[:, key_idx, 2], label=f'{subject}', color=color, linewidth=linewidth)
            ax_world_3d_2.plot(world_3d[:, key_idx, 0], world_3d[:, key_idx, 1], world_3d[:, key_idx, 2], label=f'{subject}', color=color, linewidth=linewidth)
        
        
handles, labels = ax_img_2d.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
leg = ax_img_2d.legend(by_label.values(), by_label.keys())

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(4.0)
        
# plot traj
dpi = 500
plt.sca(ax_world_3d)
for key in camera_dict.keys():
    camera_dict[key].cam_frame.draw3d()
#plt.show()
#plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_world_3d.png', dpi=dpi)

plt.sca(ax_cam_3d)
#plt.show()
plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_cam_3d.png', dpi=dpi)

plt.sca(ax_img_2d)
#plt.show()
#plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_img_2d.png', dpi=dpi)

plt.sca(ax_cam_3d_rootrel)
#plt.show()
#plt.savefig(f'{folder}/h36m_traj_analysis_{subject_list}_{keypoint_list}_cam_3d_rootrel.png', dpi=dpi)

### 3DHP

In [7]:
# Load 3dhp data
test_3dhp_gt_pkl = readpkl('3dhp_gt_test_TS1_4.pkl')
pkl = test_3dhp_gt_pkl

In [12]:
for data_type in ['train', 'test']:
    print(data_type)
    for key in pkl[data_type].keys():
        if type(pkl[data_type][key]) is np.ndarray:
            print(key, type(pkl[data_type][key]), pkl[data_type][key].shape)
        elif type(pkl[data_type][key]) is list:
            print(key, type(pkl[data_type][key]), len(pkl[data_type][key]))
        else:
            print(key, type(pkl[data_type][key]))

train
joint_2d <class 'numpy.ndarray'> (243, 17, 2)
confidence <class 'numpy.ndarray'> (243, 17)
camera_name <class 'numpy.ndarray'> (243,)
action <class 'numpy.ndarray'> (243,)
source <class 'numpy.ndarray'> (243,)
frame <class 'numpy.ndarray'> (243,)
cam_3d <class 'numpy.ndarray'> (243, 17, 3)
world_3d <class 'numpy.ndarray'> (243, 17, 3)
cam_param <class 'numpy.ndarray'> (243,)
test
joint_2d <class 'numpy.ndarray'> (24076, 17, 2)
confidence <class 'numpy.ndarray'> (24076, 17)
camera_name <class 'numpy.ndarray'> (24076,)
action <class 'numpy.ndarray'> (24076,)
source <class 'numpy.ndarray'> (24076,)
frame <class 'numpy.ndarray'> (24076,)
cam_3d <class 'numpy.ndarray'> (24076, 17, 3)
world_3d <class 'numpy.ndarray'> (24076, 17, 3)
cam_param <class 'numpy.ndarray'> (24076,)


In [8]:
test_3dhp_gt_dict = {}

for data_type in ['test']:
    sources = set(pkl[data_type]['source'])
    frames = pkl[data_type]['frame']
    for source in tqdm(sources):
        source_frames = np.where(pkl[data_type]['source'] == source)[0]
        assert is_consecutive(source_frames), f'Frames are not consecutive: {source}'
        #print(len(source_frames))
        splited = source.split('_')
        print(splited)
        subject = splited[0]
        cam_id = splited[0]
        action = splited[0]
        
        if subject not in test_3dhp_gt_dict:
            test_3dhp_gt_dict[subject] = {}
        if action not in test_3dhp_gt_dict[subject]:
            test_3dhp_gt_dict[subject][action] = {}
        if cam_id not in test_3dhp_gt_dict[subject][action]:
            test_3dhp_gt_dict[subject][action][cam_id] = {
                'joint_2d': pkl[data_type]['joint_2d'][source_frames],
                'confidence': pkl[data_type]['confidence'][source_frames],
                'world_3d': pkl[data_type]['world_3d'][source_frames],
                'cam_3d': pkl[data_type]['cam_3d'][source_frames],
                'cam_param': pkl[data_type]['cam_param'][source_frames[0]],
                'frames': source_frames
            }            

100%|██████████| 4/4 [00:00<00:00, 186.85it/s]

['TS3']
['TS1']
['TS2']
['TS4']


In [11]:
W, H = 2048, 2048

fig = plt.figure(2, figsize=(20, 10))
fig.clear()
azim = -90
elev = -90
ax1 = axes_3d(fig, loc=locs[0], zlim=(0,5), view=(-90, -90)) # cam frame
ax2 = axes_3d(fig, loc=locs[1], view=(0, 90), xlim=(-5, 4), zlim=(-4, 4)) # world frame
ax3 = axes_2d(fig, loc=locs[2], W=W, H=H) # image frame
ax4 = fig.add_subplot(locs[3], xlim=(-2, 2), ylim=(0, 8)) # x, z
ax5 = fig.add_subplot(locs[4], xlim=(-2, 2), ylim=(2, -2)) # x, y
ax6 = fig.add_subplot(locs[5], xlim=(8, 0), ylim=(2, -2)) # y, z

ax4.set_aspect('equal')
ax4.set_xlabel('x')
ax4.set_ylabel('z')

ax5.set_aspect('equal')
ax5.set_xlabel('x')
ax5.set_ylabel('y')

ax6.set_aspect('equal')
ax6.set_xlabel('z')
ax6.set_ylabel('y')

ax1.set_title(f'wrt cam_frame')
ax2.set_title(f'wrt world_frame')
ax3.set_title(f'wrt image frame')
ax4.set_title(f'x-z plane')
ax5.set_title(f'x-y plane')
ax6.set_title(f'z-y plane')


linewidth_2d = 0.4

for subject in test_3dhp_gt_dict.keys():
    for keypoint in ['pelvis', 'l_ankle', 'r_ankle', 'l_wrist', 'r_wrist']:
        print(subject, keypoint)
        keypoint_idx = get_h36m_keypoint_index(keypoint)

        clear_axes([ax1, ax2, ax3, ax4, ax5, ax6])

        action_list = list(test_3dhp_gt_dict[subject].keys())
        camera_dict = {}
        for action in action_list:
            cam_list = list(test_3dhp_gt_dict[subject][action].keys())
            for cam_id, color in zip(cam_list, ['r', 'b', 'k', 'y']):
                joint_2d = test_3dhp_gt_dict[subject][action][cam_id]['joint_2d']
                cam_3d = test_3dhp_gt_dict[subject][action][cam_id]['cam_3d']
                keypoint_traj_3d = cam_3d[:, keypoint_idx, :]
                keypoint_traj_2d = joint_2d[:, keypoint_idx, :]
                
                draw_trajectory(ax1, keypoint_traj_3d, -1, color=color) # cam frame
                ax3.plot(keypoint_traj_2d[:, 0], keypoint_traj_2d[:, 1], color=color, linewidth=linewidth_2d, label=f'{cam_id}') # image frame
                ax4.plot(keypoint_traj_3d[:, 0], keypoint_traj_3d[:, 2], color=color, linewidth=linewidth_2d, label=f'{cam_id}') # cam frame x, z
                ax5.plot(keypoint_traj_3d[:, 0], keypoint_traj_3d[:, 1], color=color, linewidth=linewidth_2d, label=f'{cam_id}') # cam frame x, y
                ax6.plot(keypoint_traj_3d[:, 2], keypoint_traj_3d[:, 1], color=color, linewidth=linewidth_2d, label=f'{cam_id}') # cam frame z, y
                
                # generate camera frame
                if cam_id not in camera_dict:
                    cam_param = test_3dhp_gt_dict[subject][action][cam_id]['cam_param'].tolist()
                    intrinsic = cam_param['intrinsic']
                    extrinsic = cam_param['extrinsic']
                    R = extrinsic[:3, :3]
                    t = extrinsic[:3, 3]
                    C = -R.T @ t
                    camera = Camera(origin=C, 
                                    calib_mat=intrinsic, 
                                    cam_default_R=R, 
                                    IMAGE_HEIGHT=1000, 
                                    IMAGE_WIDTH=1000,
                                    cam_name=cam_id)
                    camera_dict[cam_id] = camera
                    plt.sca(ax2)
                    camera.cam_frame.draw3d()
                    
            world_3d = test_3dhp_gt_dict[subject][action][cam_id]['world_3d']    
            pelvis_traj = world_3d[:, keypoint_idx, :]
            draw_trajectory(ax2, pelvis_traj, -1) # world frame
                
        handles, labels = ax4.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        leg = ax4.legend(by_label.values(), by_label.keys())
    
        # change the line width for the legend
        for line in leg.get_lines():
            line.set_linewidth(4.0)

        plt.suptitle(f'{subject}_{keypoint}')

        #plt.show()
        plt.savefig(f'/home/hrai/codes/MotionBERT/custom_codes/Dataset_analysis/3dhp_test_traj_analysis_{subject}_{keypoint}.png', dpi=600)
        #break
    #break

TS3 pelvis
TS3 l_ankle
TS3 r_ankle
TS3 l_wrist
TS3 r_wrist
TS1 pelvis
TS1 l_ankle
TS1 r_ankle
TS1 l_wrist
TS1 r_wrist
TS2 pelvis
TS2 l_ankle
TS2 r_ankle
TS2 l_wrist
TS2 r_wrist
TS4 pelvis
TS4 l_ankle
TS4 r_ankle
TS4 l_wrist
TS4 r_wrist


### Calculate distance

In [ ]:
import numpy as np
from scipy.spatial import distance

def mahalanobis_distance(cluster1, cluster2, regularization=1e-10):
    """
    Calculate the Mahalanobis distance between the centroids of two clusters.
    
    Parameters:
    - cluster1: np.ndarray, shape (n_samples1, n_features)
    - cluster2: np.ndarray, shape (n_samples2, n_features)
    - regularization: float, small value to add to the diagonal of the covariance matrix if it is singular
    
    Returns:
    - float: Mahalanobis distance between the two cluster centroids
    """
    # Calculate the centroids of each cluster
    centroid1 = np.mean(cluster1, axis=0)
    centroid2 = np.mean(cluster2, axis=0)
    
    # Calculate the covariance matrix of the combined data
    combined_data = np.vstack((cluster1, cluster2))
    cov_matrix = np.cov(combined_data.T)
    
    # Check if the covariance matrix is singular
    try:
        inv_cov_matrix = np.linalg.inv(cov_matrix)
    except np.linalg.LinAlgError:
        # Add a small value to the diagonal elements if the covariance matrix is singular
        cov_matrix += np.eye(cov_matrix.shape[0]) * regularization
        inv_cov_matrix = np.linalg.inv(cov_matrix)
    
    # Calculate the Mahalanobis distance
    dist = distance.mahalanobis(centroid1, centroid2, inv_cov_matrix)
    
    return dist

# Example usage:
cluster1_2d = np.array([[1, 2], [2, 3], [3, 4]])
cluster2_2d = np.array([[5, 6], [6, 7], [7, 8]])

cluster1_3d = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])
cluster2_3d = np.array([[5, 6, 7], [6, 7, 8], [7, 8, 9]])

print("Mahalanobis distance (2D):", mahalanobis_distance(cluster1_2d, cluster2_2d))
print("Mahalanobis distance (3D):", mahalanobis_distance(cluster1_3d, cluster2_3d))

Mahalanobis distance (2D): 1.690309799059591
Mahalanobis distance (3D): 1.6903007718210241
